In [1]:
!pip install -q gdown

# kalian download semua file dari gdrive dibawah lalu buat folder (ex. midterm_folder)
!gdown --folder 1JvI5xhPfN3VmjpWYZk9fCHG41xG697um -O midterm_folder

Retrieving folder contents
Processing file 1ALveidCmKzk4p_liQ7b663BXL0PcTaFN test_transaction.csv
Processing file 1Ujh37a1kTarNf3dSOSrj-hKDswULebYA train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1ALveidCmKzk4p_liQ7b663BXL0PcTaFN

but Gdown can't. Please check connections and permissions.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import polars as pl

BASE_PATH = "/content/drive/MyDrive/DL"

train_transaction = pl.read_csv(f"/content/drive/MyDrive/DL/train_transaction.csv")
test_transaction  = pl.read_csv(f"/content/drive/MyDrive/DL/test_transaction.csv")

print(train_transaction.shape) #590540 row dan 393 feature + 1 target
print(test_transaction.shape) #506691 row dan 393 feature

(590540, 394)
(506691, 393)


In [4]:
train_transaction.head() #isFraud adalah target featurenya

TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2987000,0,86400,68.5,"""W""",13926,null,150.0,"""discover""",142.0,"""credit""",315.0,87.0,19.0,null,null,null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,null,13.0,null,null,null,…,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987001,0,86401,29.0,"""W""",2755,404.0,150.0,"""mastercard""",102.0,"""credit""",325.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987002,0,86469,59.0,"""W""",4663,490.0,150.0,"""visa""",166.0,"""debit""",330.0,87.0,287.0,null,"""outlook.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,null,null,0.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987003,0,86499,50.0,"""W""",18132,567.0,150.0,"""mastercard""",117.0,"""debit""",476.0,87.0,null,null,"""yahoo.com""",null,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,null,…,0.0,0.0,1.0,50.0,1758.0,925.0,0.0,354.0,0.0,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2987004,0,86506,50.0,"""H""",4497,514.0,150.0,"""mastercard""",102.0,"""credit""",420.0,87.0,null,null,"""gmail.com""",null,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,null,null,null,null,null,…,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
import polars as pl
import gc

print("⏳ LANGKAH 1: Load Identity & Merge...")

# 1. Load Data Identity
# (Pastikan path folder 'midterm_folder' sesuai dengan tempat file kamu berada)
base_path = "/content/drive/MyDrive/DL"
train_id = pl.read_csv(f"/content/drive/MyDrive/DL/train_transaction.csv", ignore_errors=True)
test_id  = pl.read_csv(f"/content/drive/MyDrive/DL/test_transaction.csv", ignore_errors=True)

# 2. Gabungkan (Merge) ke data Transaksi yang sudah ada di memori
# Kita gunakan 'left join' agar semua data transaksi tetap ada
df_train = train_transaction.join(train_id, on='TransactionID', how='left')
df_test  = test_transaction.join(test_id, on='TransactionID', how='left')

# 3. Bersihkan memori (Hapus variabel lama)
del train_transaction, test_transaction, train_id, test_id
gc.collect()

print("✅ Merge Selesai.")
print(f"Total Baris Train: {df_train.height}")
print(f"Total Kolom Train: {df_train.width} (Harusnya sekitar 434)")

⏳ LANGKAH 1: Load Identity & Merge...
✅ Merge Selesai.
Total Baris Train: 590540
Total Kolom Train: 787 (Harusnya sekitar 434)


In [8]:
print("✂️ LANGKAH 2: Sampling Data...")

# Kita ambil 100.000 data saja (Wajib karena kolom ada 787, kalau semua bisa crash)
df_train = df_train.sample(n=100000, seed=42)
df_test  = df_test.sample(n=50000, seed=42)

print("✅ Sampling Selesai.")
print(f"Sisa Baris Train: {df_train.height}")
print(f"Sisa Baris Test : {df_test.height}")

✂️ LANGKAH 2: Sampling Data...
✅ Sampling Selesai.
Sisa Baris Train: 100000
Sisa Baris Test : 50000


In [9]:
import numpy as np

print("🚀 LANGKAH 3: Preprocessing...")

# 1. Pisahkan Target (isFraud) dan Fitur
y_train = df_train['isFraud']
X_train = df_train.drop(['isFraud', 'TransactionID', 'TransactionDT'])
X_test  = df_test.drop(['TransactionID', 'TransactionDT'])

# 2. Samakan Kolom (Ambil irisan kolom yang ada di Train & Test)
# Biar tidak error kalau ada kolom yang hilang di salah satu file
common_cols = [col for col in X_train.columns if col in X_test.columns]
X_train = X_train.select(common_cols)
X_test  = X_test.select(common_cols)

# 3. Encoding (Ubah Teks -> Angka)
cat_cols = [col for col in X_train.columns if X_train[col].dtype == pl.Utf8]
for col in cat_cols:
    X_train = X_train.with_columns(pl.col(col).cast(pl.Categorical).to_physical().fill_null(-1))
    X_test  = X_test.with_columns(pl.col(col).cast(pl.Categorical).to_physical().fill_null(-1))

# 4. Isi Data Kosong dengan -1
X_train = X_train.fill_null(-1)
X_test  = X_test.fill_null(-1)

# 5. Konversi ke Numpy (Float32) & Hapus Data Lama
X_train_np = X_train.to_numpy().astype(np.float32)
y_train_np = y_train.to_numpy().astype(np.float32)
X_test_np  = X_test.to_numpy().astype(np.float32)

del df_train, df_test, X_train, y_train, X_test
gc.collect()

print("✅ Preprocessing Selesai!")
print(f"Dimensi Input: {X_train_np.shape}")

🚀 LANGKAH 3: Preprocessing...
✅ Preprocessing Selesai!
Dimensi Input: (100000, 783)


In [10]:
from sklearn.preprocessing import StandardScaler

print("⚖️ LANGKAH 4: Scaling Data...")

# Inisialisasi Scaler
scaler = StandardScaler()

# Lakukan Scaling
# Fit hanya pada data Train agar tidak 'curang' (Data Leakage)
X_train_np = scaler.fit_transform(X_train_np)
X_test_np  = scaler.transform(X_test_np)

print("✅ Scaling Selesai. Data siap masuk ke Neural Network.")

⚖️ LANGKAH 4: Scaling Data...
✅ Scaling Selesai. Data siap masuk ke Neural Network.


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

print("🧠 LANGKAH 5: Build & Train Neural Network...")

# 1. Reset Session (Biar bersih dari sisa model lama jika ada)
tf.keras.backend.clear_session()

# 2. Arsitektur Model (Deep Learning)
model = models.Sequential([
    # Input Layer (Sesuai jumlah kolom data setelah scaling)
    layers.Input(shape=(X_train_np.shape[1],)),

    # Layer 1
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(), # Menstabilkan training
    layers.Dropout(0.3),         # Mencegah overfitting

    # Layer 2
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Layer 3
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Output Layer (1 Neuron, Sigmoid untuk probabilitas 0-1)
    layers.Dense(1, activation='sigmoid')
])

# 3. Hitung Class Weight (WAJIB untuk data Imbalance seperti Fraud)
# Kita beri bobot lebih besar pada kelas '1' (Fraud)
neg, pos = np.bincount(y_train_np.astype(int))
total = neg + pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"⚖️ Class Weights: {class_weight}")

# 4. Compile & Train
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

history = model.fit(
    X_train_np, y_train_np,
    epochs=10,
    batch_size=2048,      # Batch besar biar cepat di GPU
    validation_split=0.2, # 20% data latih dipakai validasi
    class_weight=class_weight,
    verbose=1
)

print("🎉 Training Selesai!")

🧠 LANGKAH 5: Build & Train Neural Network...
⚖️ Class Weights: {0: np.float64(0.5183925683241405), 1: np.float64(14.092446448703495)}
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 163ms/step - AUC: 0.6893 - loss: 0.7568 - val_AUC: 0.7654 - val_loss: 0.7460
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.7822 - loss: 0.5707 - val_AUC: 0.8057 - val_loss: 0.5906
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8087 - loss: 0.5326 - val_AUC: 0.8297 - val_loss: 0.4807
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - AUC: 0.8308 - loss: 0.5084 - val_AUC: 0.8356 - val_loss: 0.4378
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - AUC: 0.8437 - loss: 0.4907 - val_AUC: 0.8408 - val_loss: 0.4463
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8499 - loss: 0.4847 - val_AUC: 0.8393 - val_loss: 0.4751
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8460 - loss: 0.4902 - val_AUC: 0.8462 - val_loss: 0.4586
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step